In [1]:
# Set up Spark

from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("SCADA-Forecasting")
    .config("spark.driver.memory", "8g")
    .config("spark.executor.memory", "8g")
    .config("spark.driver.maxResultSize", "2g")
    .config("spark.sql.shuffle.partitions", "50")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .getOrCreate()
)

# Load Data

import pandas as pd
from sktime.forecasting.model_selection import temporal_train_test_split

pdf = pd.read_parquet(r"scada_prepro.parquet")
pdf_turbine1 = pdf[pdf["item_id"] == "1_Kelmarsh"].copy()
pdf_turbine1 = pdf_turbine1.drop(columns=["item_id"])
pdf_turbine1["timestamp"] = pd.to_datetime(pdf_turbine1["timestamp"])
pdf_turbine1 = pdf_turbine1.set_index("timestamp").sort_index()
pdft = pdf_turbine1[3:]
pdf_turbine1_no_NaN = pdf_turbine1.dropna(axis=1)
pdf_turbine1_no_NaN = pdf_turbine1_no_NaN.asfreq("10min")
pdf_turbine1_no_NaN["target"] = pdf_turbine1_no_NaN["target"].interpolate("time").ffill().bfill()
X_cols = [c for c in pdf_turbine1_no_NaN.columns if c != "target"]
pdf_turbine1_no_NaN[X_cols] = pdf_turbine1_no_NaN[X_cols].interpolate("time").ffill().bfill()


train_pdf, test_pdf = temporal_train_test_split(pdf_turbine1_no_NaN, test_size=0.2)
y_train = train_pdf["target"]

y_test = test_pdf["target"]
X_test= test_pdf.drop(columns=["target"])

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/15 23:59:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import sys
sys.path.append('/workspaces/amos2025ws03-rtdip-timeseries-forecasting/src/sdk/python')
from rtdip_sdk.pipelines.forecasting.spark.catboost_timeseries import CatboostTimeSeries

In [3]:
cbts = CatboostTimeSeries(target_col="target")

In [4]:
cbts.train(spark.createDataFrame(train_pdf))

0:	learn: 648.1804997	total: 841ms	remaining: 3m 29s
1:	learn: 619.1597785	total: 1.53s	remaining: 3m 9s
2:	learn: 591.5538326	total: 2.21s	remaining: 3m 1s
3:	learn: 565.3112375	total: 2.9s	remaining: 2m 58s
4:	learn: 540.3151533	total: 3.61s	remaining: 2m 57s
5:	learn: 516.3928463	total: 4.36s	remaining: 2m 57s
6:	learn: 494.2503217	total: 5.09s	remaining: 2m 56s
7:	learn: 472.9118067	total: 5.84s	remaining: 2m 56s
8:	learn: 452.5242805	total: 6.58s	remaining: 2m 56s
9:	learn: 433.5954722	total: 7.32s	remaining: 2m 55s
10:	learn: 415.5698927	total: 8.04s	remaining: 2m 54s
11:	learn: 398.5450693	total: 8.77s	remaining: 2m 53s
12:	learn: 382.6125773	total: 9.47s	remaining: 2m 52s
13:	learn: 367.4884276	total: 10.2s	remaining: 2m 51s
14:	learn: 352.8734485	total: 10.9s	remaining: 2m 50s
15:	learn: 339.3263482	total: 11.5s	remaining: 2m 48s
16:	learn: 326.3954555	total: 12.2s	remaining: 2m 46s
17:	learn: 314.1838754	total: 12.8s	remaining: 2m 45s
18:	learn: 302.7153239	total: 13.5s	remai

In [5]:
spark_test = spark.createDataFrame(test_pdf)
metrics = cbts.evaluate(spark_test)

NotImplementedError: RecursiveTabularRegressionForecaster can not perform in-sample prediction. Found fh with in sample index: ForecastingHorizon(start=0, stop=10541, step=1, is_relative=False)